# work in progress...

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import catboost as catb
%matplotlib inline

from tqdm import tqdm
from scipy.sparse import csr_matrix
from implicit import als
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('raw_data/retail_train.csv')
item_features = pd.read_csv('raw_data/product.csv')
user_features = pd.read_csv('raw_data/hh_demographic.csv')
transaction = pd.read_csv('raw_data/transaction_data.csv')
test = pd.read_csv('raw_data/retail_test1.csv')

In [3]:
train

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0


In [4]:
item_features

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ
...,...,...,...,...,...,...,...
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,


In [5]:
user_features

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16
...,...,...,...,...,...,...,...,...
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498


In [6]:
transaction

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2595727,1598,42305362535,711,92130,1,0.99,3228,0.00,1520,102,0.0,0.0
2595728,1598,42305362535,711,114102,1,8.89,3228,0.00,1520,102,0.0,0.0
2595729,1598,42305362535,711,133449,1,6.99,3228,0.00,1520,102,0.0,0.0
2595730,1598,42305362535,711,6923644,1,4.50,3228,-0.49,1520,102,0.0,0.0


In [7]:
test

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
88729,98,41944918665,684,945779,2,2.00,421,0.0,1705,98,0.0,0.0
88730,98,41944918665,684,993617,2,2.00,421,0.0,1705,98,0.0,0.0
88731,98,41944918665,684,1128647,2,2.00,421,0.0,1705,98,0.0,0.0
88732,98,41944918665,684,9526886,2,0.60,421,0.0,1705,98,0.0,0.0


In [8]:
user_buyses = train.groupby('user_id')['item_id'].unique().reset_index()
user_buyses.columns=['user_id', 'actual']
user_buyses.head()

,user_id,actual
0,1,"[825123, 831447, 840361, 845307, 852014, 85498..."
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55..."
2,3,"[866211, 878996, 882830, 904360, 921345, 93194..."
3,4,"[836163, 857849, 877523, 878909, 883932, 89142..."
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 69..."


In [9]:
rec_model = MainRecommender(train, weighting=True)
user_item_matrix = rec_model.user_item_matrix

In [11]:
k=5

rec_model.fit(user_item_matrix, n_factors=30, regularization=0.001, iterations=20, num_threads=4)
recommendations = [rec_model.get_similar_items_recommendation(user, N=5) for user in user_buyses["user_id"]]
precision = [precision_at_k(rec_list, bought_list, k=k) for rec_list, bought_list \
                          in zip(recommendations, user_buyses["actual"].values)]

recall = [recall_at_k(rec_list, bought_list, k=k) for rec_list, bought_list \
                          in zip(recommendations, user_buyses["actual"].values)]

In [17]:
recommendations

[[5563739, 5978648, 1077555, 871754, 826249],
 [1043590, 8090509, 7147145, 7025275, 985999],
 [885795, 856772, 865528, 1134522, 1048462],
 [865528, 6391551, 991558, 1048462, 964765],
 [972191, 12427400, 1058997, 5995589, 820895],
 [5978648, 874149, 990981, 845208, 937292],
 [5978648, 951190, 1131974, 1015247, 7147145],
 [5978648, 924352, 876504, 7025275, 830582],
 [859075, 1005902, 1098066, 991058, 13115207],
 [7147145, 1012873, 931532, 1039589, 860849],
 [1077555, 1119761, 1138434, 1133249, 830582],
 [1089954, 1004906, 1023729, 5978648, 10456196],
 [897125, 859075, 1077555, 5978648, 5978648],
 [897125, 1004906, 931532, 865528, 859075],
 [5978648, 901062, 859075, 1077555, 1037863],
 [2445642, 2417516, 2496543, 2495324, 2445642],
 [1022510, 897125, 1054889, 1077659, 846743],
 [903255, 1098066, 991223, 861272, 12524651],
 [897125, 1048462, 859075, 1098066, 985119],
 [880888, 1058997, 1128363, 1128363, 911724],
 [845288, 859075, 897125, 949006, 856326],
 [5978648, 901062, 824323, 897125, 

In [14]:
res

{'10': ([0.4,
   0.4,
   0.2,
   0.0,
   0.2,
   0.6,
   0.4,
   0.4,
   0.2,
   0.2,
   0.0,
   0.0,
   0.6,
   0.2,
   0.2,
   0.8,
   0.2,
   0.4,
   1.0,
   0.6,
   0.0,
   0.8,
   0.4,
   0.0,
   0.6,
   0.4,
   0.6,
   0.4,
   0.8,
   0.0,
   0.4,
   0.4,
   0.2,
   0.0,
   0.4,
   0.2,
   0.0,
   0.2,
   0.2,
   0.4,
   0.2,
   0.4,
   0.0,
   0.0,
   0.4,
   0.4,
   0.0,
   0.4,
   0.6,
   0.2,
   0.2,
   0.4,
   0.4,
   0.0,
   0.2,
   0.8,
   0.4,
   0.8,
   0.6,
   0.2,
   0.0,
   1.0,
   0.2,
   0.4,
   0.2,
   0.6,
   0.2,
   0.8,
   1.0,
   0.2,
   0.4,
   0.4,
   0.4,
   0.8,
   1.0,
   0.2,
   0.4,
   0.8,
   0.6,
   0.8,
   0.6,
   0.0,
   0.2,
   0.2,
   0.4,
   0.0,
   0.6,
   0.6,
   0.0,
   0.0,
   0.4,
   0.4,
   0.6,
   0.2,
   0.6,
   0.0,
   0.8,
   0.4,
   0.6,
   0.2,
   0.0,
   0.0,
   0.0,
   0.2,
   0.6,
   0.2,
   0.4,
   0.0,
   0.2,
   0.8,
   0.2,
   0.0,
   0.6,
   0.2,
   0.8,
   0.2,
   0.4,
   0.4,
   0.2,
   0.4,
   0.6,
   0.0,
   0.8,
   0.0,
  

In [ ]:
# item_features.columns = [col.lower() for col in item_features.columns]
# user_features.columns = [col.lower() for col in user_features.columns]

# item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
# user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# val_lvl_1_size_weeks = 6
# val_lvl_2_size_weeks = 3

# data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
# data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
#                       (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

# data_train_lvl_2 = data_val_lvl_1.copy()
# data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

# data_train_lvl_1.head(2)